<a href="https://colab.research.google.com/github/hassanSattariNia/FederatedLearning/blob/main/ALBERT_SPLIT_VERSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install transformers datasets torch evaluate

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This be

In [3]:
from datasets import load_dataset
import evaluate


accuracy_metric = evaluate.load("accuracy")
dataset = load_dataset("glue", "mrpc")

In [4]:
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

Train set size: 3668
Validation set size: 408
Test set size: 1725


In [6]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)

    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Directory to save checkpoints and logs
    evaluation_strategy="epoch",
     learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics = compute_metrics
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.392271,0.833333,0.887789
2,No log,0.388419,0.840686,0.891122
3,0.377900,0.479466,0.875000,0.907776
4,0.377900,0.663617,0.860294,0.898032
5,0.133400,0.805557,0.857843,0.897887


KeyboardInterrupt: 

In [9]:
results = trainer.evaluate()
print(results)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.392271,0.833333,0.887789
2,No log,0.388419,0.840686,0.891122
3,0.377900,0.479466,0.875000,0.907776
4,0.377900,0.663617,0.860294,0.898032
5,0.133400,0.847997,0.875000,0.910683


{'eval_loss': 0.8479965925216675, 'eval_accuracy': 0.875, 'eval_f1': 0.9106830122591943}


In [10]:
print(model)

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [12]:
import torch
from transformers import AlbertTokenizer, AlbertForSequenceClassification
from datasets import load_dataset

# بارگذاری مدل و توکنایزر
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2")
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

# بارگذاری دیتاست GLUE MRPC
dataset = load_dataset("glue", "mrpc")
sample = dataset["train"][0]

# توکنایز کردن نمونه
inputs = tokenizer(
    sample["sentence1"], sample["sentence2"],
    return_tensors="pt", padding=True, truncation=True
)

# اجرای بخش اول: امبدینگ
with torch.no_grad():
    embeddings = model.albert.embeddings(inputs["input_ids"], inputs["token_type_ids"])

# چاپ شکل خروجی و ذخیره آن در متغیر کمکی
output_part1 = embeddings
print("Output of the embedding layer:", output_part1.shape)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Output of the embedding layer: torch.Size([1, 59, 128])


In [13]:
# اجرای بخش دوم: لایه مپینگ امبدینگ به فضای 768 بُعدی
with torch.no_grad():
    output_part2 = model.albert.encoder.embedding_hidden_mapping_in(output_part1)

# چاپ شکل خروجی
print("Output of the embedding hidden mapping layer:", output_part2.shape)


Output of the embedding hidden mapping layer: torch.Size([1, 59, 768])


In [15]:
# اجرای بخش سوم: لایه‌های ترانسفورمر
with torch.no_grad():
    output_part3 = model.albert.encoder.albert_layer_groups[0].albert_layers[0](output_part2)

# چاپ شکل خروجی
print("Output of the transformer layer:", output_part3[0].shape)


Output of the transformer layer: torch.Size([1, 59, 768])


In [16]:
# اجرای بخش چهارم: لایه Pooler
with torch.no_grad():
    pooled_output = model.albert.pooler(output_part3[0][:, 0, :])  # انتخاب اولین توکن (CLS) برای pooler
    output_part4 = model.albert.pooler_activation(pooled_output)

# چاپ شکل خروجی
print("Output of the pooler layer:", output_part4.shape)


Output of the pooler layer: torch.Size([1, 768])


In [17]:
# اجرای بخش نهایی: لایه طبقه‌بندی‌کننده
with torch.no_grad():
    logits = model.classifier(output_part4)

# چاپ شکل خروجی
print("Output of the classifier layer (logits):", logits.shape)
print("Logits:", logits)


Output of the classifier layer (logits): torch.Size([1, 2])
Logits: tensor([[ 0.0044, -0.4695]])


In [23]:
from transformers import AdamW
import evaluate
import torch

# تنظیمات بهینه‌سازی
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

# متریک‌های ارزیابی
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# حلقه آموزش
epochNumber = 0
for epoch in range(num_epochs):
    epochNumber += 1
    print(f"epoch : {epochNumber}")
    model.train()
    total_loss = 0
    batchNumber = 0
    for batch in dataset["train"]:
        batchNumber += 1
        print(batchNumber, end=" ")
        inputs = tokenizer(
            batch["sentence1"], batch["sentence2"],
            return_tensors="pt", padding=True, truncation=True
        )

        # اجرای بخش‌های مدل به صورت مرحله به مرحله
        with torch.no_grad():
            output_part1 = model.albert.embeddings(inputs["input_ids"], inputs["token_type_ids"])
            output_part2 = model.albert.encoder.embedding_hidden_mapping_in(output_part1)
            output_part3 = model.albert.encoder.albert_layer_groups[0].albert_layers[0](output_part2)[0]
            pooled_output = model.albert.pooler(output_part3[:, 0, :])
            output_part4 = model.albert.pooler_activation(pooled_output)

        # محاسبه logits و loss
        logits = model.classifier(output_part4)
        loss_fn = torch.nn.CrossEntropyLoss()
        labels = torch.tensor([batch["label"]])  # تبدیل label به tensor
        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        # به‌روزرسانی گرادیان و تنظیم پارامترها
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(dataset["train"])

    # حلقه ارزیابی
    model.eval()
    total_eval_loss = 0
    for batch in dataset["validation"]:
        inputs = tokenizer(
            batch["sentence1"], batch["sentence2"],
            return_tensors="pt", padding=True, truncation=True
        )

        # اجرای مدل به صورت مرحله به مرحله در حالت ارزیابی
        with torch.no_grad():
            output_part1 = model.albert.embeddings(inputs["input_ids"], inputs["token_type_ids"])
            output_part2 = model.albert.encoder.embedding_hidden_mapping_in(output_part1)
            output_part3 = model.albert.encoder.albert_layer_groups[0].albert_layers[0](output_part2)[0]
            pooled_output = model.albert.pooler(output_part3[:, 0, :])
            output_part4 = model.albert.pooler_activation(pooled_output)
            logits = model.classifier(output_part4)

            # محاسبه loss و متریک‌ها
            labels = torch.tensor([batch["label"]])  # تبدیل label به tensor
            loss = loss_fn(logits, labels)
            total_eval_loss += loss.item()

            predictions = torch.argmax(logits, dim=-1)
            accuracy_metric.add_batch(predictions=predictions, references=[batch["label"]])
            f1_metric.add_batch(predictions=predictions, references=[batch["label"]])

    avg_eval_loss = total_eval_loss / len(dataset["validation"])
    eval_accuracy = accuracy_metric.compute()["accuracy"]
    eval_f1 = f1_metric.compute(average="weighted")["f1"]

    print({
        'epoch': epoch + 1,
        'train_loss': avg_train_loss,
        'eval_loss': avg_eval_loss,
        'eval_accuracy': eval_accuracy,
        'eval_f1': eval_f1
    })


epoch : 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 27

KeyboardInterrupt: 

In [25]:
from torch.utils.data import DataLoader

# تنظیمات
batch_size = 16  # اندازه batch دلخواه

# ایجاد DataLoader ها
train_dataloader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(dataset["validation"], batch_size=batch_size)


In [29]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
import evaluate
from torch.utils.data import DataLoader

# تنظیمات اولیه
batch_size = 16
num_epochs = 10
learning_rate = 2e-5

# ایجاد DataLoader ها
train_dataloader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(dataset["validation"], batch_size=batch_size)

# تنظیمات بهینه‌سازی و scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# متریک‌های ارزیابی
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# حلقه آموزش
for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()
    total_loss = 0
    for batchNumber, batch in enumerate(train_dataloader, 1):
        print(f"Batch {batchNumber}", end=" ")

        # پردازش ورودی‌ها
        inputs = tokenizer(
            batch["sentence1"], batch["sentence2"],
            return_tensors="pt", padding=True, truncation=True
        )

        # مراحل پردازش مدل به صورت بخش بخش (بدون no_grad برای محاسبه گرادیان)
        output_part1 = model.albert.embeddings(inputs["input_ids"], inputs["token_type_ids"])
        output_part2 = model.albert.encoder.embedding_hidden_mapping_in(output_part1)
        output_part3 = model.albert.encoder.albert_layer_groups[0].albert_layers[0](output_part2)[0]
        pooled_output = model.albert.pooler(output_part3[:, 0, :])
        output_part4 = model.albert.pooler_activation(pooled_output)

        # محاسبه logits و loss
        logits = model.classifier(output_part4)
        loss_fn = torch.nn.CrossEntropyLoss()
        labels = torch.tensor(batch["label"])  # تبدیل label به tensor با ابعاد درست
        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        # به‌روزرسانی گرادیان و تنظیم پارامترها
        loss.backward()  # محاسبه گرادیان‌ها با استفاده از loss
        optimizer.step()  # به‌روزرسانی پارامترهای مدل
        scheduler.step()  # به‌روزرسانی نرخ یادگیری با استفاده از scheduler
        optimizer.zero_grad()  # صفر کردن گرادیان‌ها برای batch بعدی

    avg_train_loss = total_loss / len(train_dataloader)

    # حلقه ارزیابی
    model.eval()
    total_eval_loss = 0
    for batch in eval_dataloader:
        inputs = tokenizer(
            batch["sentence1"], batch["sentence2"],
            return_tensors="pt", padding=True, truncation=True
        )

        # پردازش مدل به صورت بخش بخش در حالت ارزیابی
        with torch.no_grad():
            output_part1 = model.albert.embeddings(inputs["input_ids"], inputs["token_type_ids"])
            output_part2 = model.albert.encoder.embedding_hidden_mapping_in(output_part1)
            output_part3 = model.albert.encoder.albert_layer_groups[0].albert_layers[0](output_part2)[0]
            pooled_output = model.albert.pooler(output_part3[:, 0, :])
            output_part4 = model.albert.pooler_activation(pooled_output)
            logits = model.classifier(output_part4)

            # محاسبه loss و متریک‌ها
            labels = torch.tensor(batch["label"])
            loss = loss_fn(logits, labels)
            total_eval_loss += loss.item()

            predictions = torch.argmax(logits, dim=-1)
            accuracy_metric.add_batch(predictions=predictions, references=labels)
            f1_metric.add_batch(predictions=predictions, references=labels)

    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    eval_accuracy = accuracy_metric.compute()["accuracy"]
    eval_f1 = f1_metric.compute(average="weighted")["f1"]

    print({
        'epoch': epoch + 1,
        'train_loss': avg_train_loss,
        'eval_loss': avg_eval_loss,
        'eval_accuracy': eval_accuracy,
        'eval_f1': eval_f1
    })


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1
Batch 1 

<ipython-input-29-427bff790e96>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"])  # تبدیل label به tensor با ابعاد درست


Batch 2 Batch 3 Batch 4 Batch 5 Batch 6 Batch 7 Batch 8 Batch 9 Batch 10 Batch 11 Batch 12 Batch 13 Batch 14 Batch 15 Batch 16 Batch 17 Batch 18 Batch 19 Batch 20 Batch 21 Batch 22 Batch 23 Batch 24 Batch 25 Batch 26 Batch 27 Batch 28 Batch 29 Batch 30 Batch 31 Batch 32 Batch 33 Batch 34 Batch 35 Batch 36 Batch 37 Batch 38 Batch 39 Batch 40 Batch 41 Batch 42 Batch 43 Batch 44 Batch 45 Batch 46 Batch 47 Batch 48 Batch 49 Batch 50 Batch 51 Batch 52 Batch 53 Batch 54 Batch 55 Batch 56 Batch 57 Batch 58 Batch 59 Batch 60 Batch 61 Batch 62 Batch 63 Batch 64 Batch 65 Batch 66 Batch 67 Batch 68 Batch 69 Batch 70 Batch 71 Batch 72 Batch 73 Batch 74 Batch 75 Batch 76 Batch 77 Batch 78 Batch 79 Batch 80 Batch 81 Batch 82 Batch 83 Batch 84 Batch 85 Batch 86 Batch 87 Batch 88 Batch 89 Batch 90 Batch 91 Batch 92 Batch 93 Batch 94 Batch 95 Batch 96 Batch 97 Batch 98 Batch 99 Batch 100 Batch 101 Batch 102 Batch 103 Batch 104 Batch 105 Batch 106 Batch 107 Batch 108 Batch 109 Batch 110 Batch 111 Batch 

<ipython-input-29-427bff790e96>:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"])


{'epoch': 1, 'train_loss': 0.5728969971770825, 'eval_loss': 0.5802988868493301, 'eval_accuracy': 0.7058823529411765, 'eval_f1': 0.6268044313476817}
Epoch: 2
Batch 1 

<ipython-input-29-427bff790e96>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"])  # تبدیل label به tensor با ابعاد درست


Batch 2 Batch 3 Batch 4 Batch 5 Batch 6 Batch 7 Batch 8 Batch 9 Batch 10 Batch 11 Batch 12 Batch 13 Batch 14 Batch 15 Batch 16 Batch 17 Batch 18 

KeyboardInterrupt: 